In [ ]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from gammapy.irf import load_cta_irfs
from gammapy.maps import WcsGeom, MapAxis
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    GaussianSpatialModel,
    SkyModel,
    Models,
    FoVBackgroundModel,
)
from gammapy.makers import MapDatasetMaker, SafeMaskMaker
from gammapy.modeling import Fit
from gammapy.data import Observation
from gammapy.datasets import MapDataset

In [ ]:
sim = ctools.ctobssim()
sim['inmodel'] = 'model_to_simulate.xml'
sim['outevents'] = 'events_' + str(command_line_argument) + '.fits'
#sim['caldb'] = 'prod3b'
#sim['irf'] = 'North_z20_average_50h'
#sim['ra'] = 98.48
#sim['dec'] = 17.77
sim['rad'] = 5.
#sim['tmin'] = 0
sim['seed'] = numpy.random.randint(low=0,high=100000000)
#sim['tmax'] = time
sim['emin'] = emin
sim['emax'] = emax

In [ ]:
ls

In [ ]:
# Loading IRFs
#irfs = load_cta_irfs("/Users/vdk/Downloads/irf_file.fits")
irfs = load_cta_irfs("/Users/vdk/WorkWithAnastasia/cta-prod5-zenodo-fitsonly-v0.1/fits/CTA-Performance-prod5-v0.1-North-20deg.FITS/Prod5.fits")

In [ ]:
irfs

In [ ]:
# Define the observation parameters (typically the observation duration and the pointing position):
livetime = 2.0 * u.hr
pointing = SkyCoord(98.48, 17.77, unit="deg", frame="icrs")

In [ ]:
# Define map geometry for binned simulation
energy_reco = MapAxis.from_edges(
    np.logspace(-1.0, 1.0, 10), unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(98.48, 17.77),
    binsz=0.02,
    width=(6, 6),
    frame="icrs",
    axes=[energy_reco],
)
# It is usually useful to have a separate binning for the true energy axis
energy_true = MapAxis.from_edges(
    np.logspace(-1.5, 1.5, 30), unit="TeV", name="energy", interp="log"
)

empty = MapDataset.create(geom, name="dataset-simu")
geom

In [ ]:
# Define sky model to used simulate the data.
# Here we use a Gaussian spatial model and a Power Law spectral model.
spatial_model = GaussianSpatialModel(
    lon_0="0.2 deg", lat_0="0.1 deg", sigma="0.3 deg", frame="galactic"
)
spectral_model = PowerLawSpectralModel(
    index=3, amplitude="1e-11 cm-2 s-1 TeV-1", reference="1 TeV"
)
model_simu = SkyModel(
    spatial_model=spatial_model,
    spectral_model=spectral_model,
    name="model-simu",
)

bkg_model = FoVBackgroundModel(dataset_name="dataset-simu")

models = Models([model_simu, bkg_model])
print(models)

In [ ]:
# Create an in-memory observation
obs = Observation.create(pointing=pointing, livetime=livetime, irfs=irfs)
print(obs)

In [ ]:
# Make the MapDataset
maker = MapDatasetMaker(selection=["exposure", "background", "psf", "edisp"])

maker_safe_mask = SafeMaskMaker(methods=["offset-max"], offset_max=4.0 * u.deg)

dataset = maker.run(empty, obs)
dataset = maker_safe_mask.run(dataset, obs)
print(dataset)

In [ ]:
# Add the model on the dataset and Poission fluctuate
dataset.models = models
dataset.fake()
# Save dataset to fits fil
e
dataset.write("observation.fits")

In [ ]:
# This is what the XML file looks like
# !tail -n 20 $CTADATA/models/models_gps.xml

# TODO: write this example!

# Read XML file and access spectrum parameters
# from gammapy.extern import xmltodict

# filename = os.path.join(os.environ["CTADATA"], "models/models_gps.xml")
# data = xmltodict.parse(open(filename).read())
# data = data["source_library"]["source"][-1]
# data = data["spectrum"]["parameter"]
# data

# Create a spectral model the the right units
# from astropy import units as u
# from gammapy.modeling.models import PowerLawSpectralModel

# par_to_val = lambda par: float(par["@value"]) * float(par["@scale"])
# spec = PowerLawSpectralModel(
#     amplitude=par_to_val(data[0]) * u.Unit("cm-2 s-1 MeV-1"),
#     index=par_to_val(data[1]),
#     reference=par_to_val(data[2]) * u.Unit("MeV"),
# )
# print(spec)

In [ ]:
from gammapy.maps import Map
from gammapy.modeling.models import (
    Models,
    PowerLawSpectralModel,
    SkyModel,
    TemplateSpatialModel,
)

filename = "/Users/vdk/WorkWithAnastasia/allenergies.fits"
m = Map.read(filename)
m = m.copy(unit="sr^-1")
model = TemplateSpatialModel(m, filename=filename)

#model.plot(add_cbar=True)

In [ ]:
from gammapy.maps import MapAxis, WcsGeom
#from gammapy.datasets import MapDataset

energy_axis = MapAxis.from_bounds(
    1, 10, nbin=11, name="energy", unit="TeV", interp="log"
)
geom = WcsGeom.create(axes=[energy_axis])
dataset_empty = MapDataset.create(geom=geom)
print(dataset_empty)

In [ ]:
a = [1,2,3,0]

In [ ]:
a[3]=3

In [ ]:
a

In [ ]:
np.logspace(-1.0, 1.0, 10)